In [32]:
#Buenos intakes:
    #https://maxgmarin.github.io/AC209a_FinalProject_EEM/notebook_Markdown/ParsingAll_AudioFeatures_120417.html
    #https://www.linkedin.com/pulse/extracting-your-fav-playlist-info-spotifys-api-samantha-jones/
    #https://towardsdatascience.com/how-to-create-large-music-datasets-using-spotipy-40e7242cc6a6
    #https://medium.com/@samlupton/spotipy-get-features-from-your-favourite-songs-in-python-6d71f0172df0

In [13]:
#https://stackoverflow.com/questions/39086287/spotipy-how-to-read-more-than-100-tracks-from-a-playlist
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import pandas as pd

In [14]:
client_id = '8e6e962237b74490aca58fd858c5a54f'
client_secret = '06fd25cc7b164f7e88927938c5c1a589'

client_credentials_manager = SpotifyClientCredentials(client_id=client_id, client_secret=client_secret)
sp = spotipy.Spotify(client_credentials_manager = client_credentials_manager)

In [24]:
# Function to extract MetaData from a playlist thats longer than 100 songs
def get_playlist_tracks_more_than_100_songs(username, playlist_id_list):
    df_big = []
    for playlist_id in playlist_id_list:
        results = sp.user_playlist_tracks(username,playlist_id)
        tracks = results['items']
        while results['next']:
            results = sp.next(results)
            tracks.extend(results['items'])
        results = tracks    

        playlist_tracks_id = []
        playlist_tracks_titles = []
        playlist_tracks_artists = []
        playlist_tracks_first_artists = []
        playlist_tracks_first_release_date = []
        playlist_tracks_popularity = []

        for i in range(len(results)):
            #print(i) # Counter
            if i == 0:
                playlist_tracks_id = results[i]['track']['id']
                playlist_tracks_titles = results[i]['track']['name']
                playlist_tracks_first_release_date = results[i]['track']['album']['release_date']
                playlist_tracks_popularity = results[i]['track']['popularity']

                artist_list = []
                for artist in results[i]['track']['artists']:
                    artist_list= artist['name']
                playlist_tracks_artists = artist_list

                features = sp.audio_features(playlist_tracks_id)
                features_df = pd.DataFrame(data=features, columns=features[0].keys())
                features_df['title'] = playlist_tracks_titles
                features_df['all_artists'] = playlist_tracks_artists
                features_df['popularity'] = playlist_tracks_popularity
                features_df['release_date'] = playlist_tracks_first_release_date
                features_df = features_df[['id', 'title', 'all_artists', 'popularity', 'release_date',
                                           'danceability', 'energy', 'key', 'loudness',
                                           'mode', 'acousticness', 'instrumentalness',
                                           'liveness', 'valence', 'tempo',
                                           'duration_ms', 'time_signature']]
                continue
            else:
                try:
                    playlist_tracks_id = results[i]['track']['id']
                    playlist_tracks_titles = results[i]['track']['name']
                    playlist_tracks_first_release_date = results[i]['track']['album']['release_date']
                    playlist_tracks_popularity = results[i]['track']['popularity']
                    artist_list = []
                    for artist in results[i]['track']['artists']:
                        artist_list= artist['name']
                    playlist_tracks_artists = artist_list
                    features = sp.audio_features(playlist_tracks_id)
                    new_row = {'id':[playlist_tracks_id],
                   'title':[playlist_tracks_titles],
                   'all_artists':[playlist_tracks_artists],
                   'popularity':[playlist_tracks_popularity],
                   'release_date':[playlist_tracks_first_release_date],
                   'danceability':[features[0]['danceability']],
                   'energy':[features[0]['energy']],
                   'key':[features[0]['key']],
                   'loudness':[features[0]['loudness']],
                   'mode':[features[0]['mode']],
                   'acousticness':[features[0]['acousticness']],
                   'instrumentalness':[features[0]['instrumentalness']],
                   'liveness':[features[0]['liveness']],
                   'valence':[features[0]['valence']],
                   'tempo':[features[0]['tempo']],
                   'duration_ms':[features[0]['duration_ms']],
                   'time_signature':[features[0]['time_signature']]
                   }

                    dfs = [features_df, pd.DataFrame(new_row)]
                    #features_df = pd.concat(dfs, ignore_index = True)
                    
                    df_big = df_big + [pd.DataFrame(new_row)]
                except:
                    continue
    features_df_big = pd.concat(df_big, ignore_index = True)       
    return features_df_big

In [56]:
df = get_playlist_tracks_more_than_100_songs('spotify', ['5B3gb8HbvFm02ccQVH33IO', '3owJApDCnuxVg9nw6fqlSC', '2PzF3urLEgnQs4cmokAVHX', '52G9ij1Yrh1dVmXGXqVemZ', '370aiUDCl2Dm6BWgyE46YE'])
df

,id,title,all_artists,popularity,release_date,danceability,energy,key,loudness,mode,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,time_signature
0,5ubvP9oKmxLUVq506fgLhk,Stayin Alive,Bee Gees,80,2020-12-11,0.702,0.773,10,-9.256,0,0.0288,0.008710,0.1470,0.953,103.564,285373,4
1,2LScqpywMqGcnum6nNaxXX,"Yes Sir, I Can Boogie",Baccara,66,1993-06-14,0.454,0.769,2,-7.653,1,0.1270,0.000003,0.1750,0.491,122.531,273867,4
2,3K7Q9PHUWPTaknlbFPThn2,Celebration,Kool & The Gang,75,1980-09-29,0.885,0.664,1,-11.795,1,0.1450,0.048300,0.0767,0.906,121.632,298867,4
3,2Foc5Q5nqNiosCNqttzHof,Get Lucky (feat. Pharrell Williams & Nile Rodg...,Nile Rodgers,81,2013-04-19,0.794,0.811,6,-8.966,0,0.0426,0.000001,0.1010,0.862,116.047,248413,4
4,2xiOdusRnZezQok1RgLNeS,"You Should Be Dancing - Edit / From ""Saturday ...",Bee Gees,70,1976-01-01,0.681,0.716,0,-10.911,1,0.0182,0.170000,0.3370,0.963,123.009,256493,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
790,4qM461TqtpnP4GLRIXwEnW,Sussudio - 2016 Remaster,Phil Collins,69,1985-01-25,0.714,0.935,5,-5.195,1,0.0529,0.093400,0.0515,0.725,121.232,263107,4
791,1ot6jEe4w4hYnsOPjd3xKQ,I'm So Excited,The Pointer Sisters,74,1982-02-01,0.590,0.868,5,-5.832,0,0.0962,0.000322,0.4420,0.798,184.348,230640,4
792,0Dm43YLUlGdePpSbI1ct8h,Part-Time Lover,Stevie Wonder,70,1985-09-13,0.662,0.624,10,-13.606,0,0.1650,0.000000,0.0674,0.961,174.513,252560,4
793,2fjJt3OQ8MPaZ3LMzAWiTA,Typical Male,Tina Turner,52,1986-01-10,0.762,0.745,1,-7.772,0,0.0172,0.013700,0.0612,0.794,106.013,256827,4


In [57]:
df.shape

(795, 17)

In [58]:
df['title'].value_counts()

Love Come Home - Radio Mix                        3
Komodo                                            3
I'm So Excited                                    3
Let's Groove                                      3
La Passion                                        3
                                                 ..
Push The Feeling On - Mk Dub Revisited Edit       1
Tainted Love                                      1
Wanna Be Startin' Somethin'                       1
Put Your Hands Up - Original Pump It Up Anthem    1
Mr. Vain - Original Radio Edit                    1
Name: title, Length: 731, dtype: int64

In [59]:
#Saving the dataset
df.to_csv('./data/mix_5.csv', index = False)

In [39]:
4N7yb3zNeXEABmS9WRPESZ (alt)

37i9dQZF1DX6XceWZP1znY (anime)

37i9dQZF1DWZBCPUIUs2iR(country)
1mJhSx6aYQmINsZ8dG4gzU (country)
4EKxAJeKAwzd0ifNsFOUYo (country)
4TgQnrfeA84A2uSBBnXPiw (country)
['37i9dQZF1DWZBCPUIUs2iR', '1mJhSx6aYQmINsZ8dG4gzU', '4EKxAJeKAwzd0ifNsFOUYo', '4TgQnrfeA84A2uSBBnXPiw']


3iWdDGK2cue86Tb7wufAFU (pop)
6ihgnlqxvYZfc5LbBKS4Bt (pop)
3k42316EoqW5fE8YjVpltu (pop)
37i9dQZF1DWXJyjYpHunCf (pop)
['3iWdDGK2cue86Tb7wufAFU', '6ihgnlqxvYZfc5LbBKS4Bt', '3k42316EoqW5fE8YjVpltu', '37i9dQZF1DWXJyjYpHunCf']


0weizyV5WNZP3tvfXWVfmg (hiphop)
3RcRK9HGTAm9eLW1LepWKZ (hiphop)
7oe9PHYHNshF3oc3X0IIdy (hiphop)
6RIngYgeaogaJlbojgSFSX (hiphop)
5DZ93TH6ABgWYRgdBNJs8O (hiphop)
1L1VsYZJ55P731V7b8gFOk (hiphop)
['0weizyV5WNZP3tvfXWVfmg', '3RcRK9HGTAm9eLW1LepWKZ', '7oe9PHYHNshF3oc3X0IIdy', '6RIngYgeaogaJlbojgSFSX', '5DZ93TH6ABgWYRgdBNJs8O', '1L1VsYZJ55P731V7b8gFOk']

37i9dQZF1DX04mASjTsvf0 (rb)
0bi6JVXt2xxjfYIr728Jsz (rb)
37i9dQZF1DX4SBhb3fqCJd (rb)
2dyKyLOiDnMJFm73ycNV4k (rb)
['37i9dQZF1DX04mASjTsvf0', '0bi6JVXt2xxjfYIr728Jsz', '37i9dQZF1DX4SBhb3fqCJd', '2dyKyLOiDnMJFm73ycNV4k']


7ABMovBcmt0PaZ9uRQ10Vv (jazz)
37i9dQZF1DXbITWG1ZJKYt (jazz)
37i9dQZF1DWV7EzJMK2FUI (jazz)
5EUhooJ7cFHScZcmfz66Qp (jazz)
37i9dQZF1DXdziGPHNE40t (jazz)
['7ABMovBcmt0PaZ9uRQ10Vv', '37i9dQZF1DXbITWG1ZJKYt', '37i9dQZF1DWV7EzJMK2FUI', '5EUhooJ7cFHScZcmfz66Qp', '37i9dQZF1DXdziGPHNE40t']

----
37i9dQZF1DXd9rSDyQguIk (blues)
5uBRWDOfL0Q0eyJ4IxdpxW (blues)
5TkTomPbQuSNDxdlWg2fCx (blues)
1i9NKaGOEQsRmR392BqMxE (blues)
1m6SO0I2Y82NCsAfmG7bVP (blues)
['37i9dQZF1DXd9rSDyQguIk', '5uBRWDOfL0Q0eyJ4IxdpxW' , '5TkTomPbQuSNDxdlWg2fCx', '1i9NKaGOEQsRmR392BqMxE', '1m6SO0I2Y82NCsAfmG7bVP']

1h0CEZCm6IbFTbxThn6Xcs (classical)
1CYNLCLR5ZH9tllxGPA4GS (classical)
6wObnEPQ63a4kei1sEcMdH (classical)
3BAtH3eZHdi4vfN6ZkpGiX (classical)
['1h0CEZCm6IbFTbxThn6Xcs', '1CYNLCLR5ZH9tllxGPA4GS', '6wObnEPQ63a4kei1sEcMdH', '3BAtH3eZHdi4vfN6ZkpGiX']


5JMihbVmqBAsdfvkVYX9Fv (latin)
2fdCOYizvfG6JoH0zGQWB8 (latin)
37i9dQZF1DX10zKzsJ2jva (latin)
29Ah5RnWJ0YKxSzUpjk1kB (latin)
00CtpUus1xPIBlHUTYo3yn (latin)
1vIZxBcvgGHE9HXbQnzXYI (latin)
['5JMihbVmqBAsdfvkVYX9Fv', '2fdCOYizvfG6JoH0zGQWB8','37i9dQZF1DX10zKzsJ2jva', '29Ah5RnWJ0YKxSzUpjk1kB', '00CtpUus1xPIBlHUTYo3yn', '1vIZxBcvgGHE9HXbQnzXYI']

----

37i9dQZF1DWV1PBrIG2weG (chill)
7ozIozDp260fjNOZy1yzRG (chill)
37i9dQZF1DX4WYpdgoIcn6 (chill)
23tyPRHQ1kwG25OIlWBCbz (chill)
6H2OZzeoaIdgL17NoF6jD5 (chill)
['37i9dQZF1DWV1PBrIG2weG', '7ozIozDp260fjNOZy1yzRG', '37i9dQZF1DX4WYpdgoIcn6', '23tyPRHQ1kwG25OIlWBCbz', '6H2OZzeoaIdgL17NoF6jD5']

2237sMNMlXS4wWLgdQ1UuV (workout)
2PAGgoyhVYdC0q6vnb8qAL (workout)
37i9dQZF1DWYp5sAHdz27Y (workout)
37i9dQZF1DX32NsLKyzScr (workout)
7mZZkjpyoY83wHbssEtzNF (workout)
['2237sMNMlXS4wWLgdQ1UuV', '2PAGgoyhVYdC0q6vnb8qAL', '37i9dQZF1DWYp5sAHdz27Y', '37i9dQZF1DX32NsLKyzScr', '7mZZkjpyoY83wHbssEtzNF']


4tuReEKVsupVoQOldfIpiu (party)
37i9dQZF1DX7e8TjkFNKWH (party)
37i9dQZF1DWWylYLMvjuRG (party)
37i9dQZF1DX7F6T2n2fegs (party)
47ipPvW8GYxDzzS7vGemJb (party)
['4tuReEKVsupVoQOldfIpiu', '37i9dQZF1DX7e8TjkFNKWH', '37i9dQZF1DWWylYLMvjuRG', '37i9dQZF1DX7F6T2n2fegs', '47ipPvW8GYxDzzS7vGemJb']

----

04EKmJhBK8dLL6SONP5dyq (dance)
37i9dQZF1DX0BcQWzuB7ZO (dance)
37i9dQZF1DX8a1tdzq5tbM (dance)
37i9dQZF1DX35X4JNyBWtb (dance)
37i9dQZF1DXccH49bh52dB (dance)
['04EKmJhBK8dLL6SONP5dyq', '37i9dQZF1DX0BcQWzuB7ZO', '37i9dQZF1DX8a1tdzq5tbM', '37i9dQZF1DX35X4JNyBWtb', '37i9dQZF1DXccH49bh52dB']


1P27ra5VqAizmkcUzVAvp2 (kids)
2NXQMbuBJHxm37uC6CArRM (kids)
2BA8cL1xUJHFXUkfBF7Hl4 (kids)
jVt6TsQXWz3xzkNcxF2Db (kids)
5FlHPnEV1ECP1Eu1en8Rtf (kids)
['1P27ra5VqAizmkcUzVAvp2', '2NXQMbuBJHxm37uC6CArRM', '2BA8cL1xUJHFXUkfBF7Hl4', 'jVt6TsQXWz3xzkNcxF2Db', '5FlHPnEV1ECP1Eu1en8Rtf']
(este no ha funcionado)

5B3gb8HbvFm02ccQVH33IO (disco)
3owJApDCnuxVg9nw6fqlSC (disco)
2PzF3urLEgnQs4cmokAVHX (disco)
52G9ij1Yrh1dVmXGXqVemZ (disco)
370aiUDCl2Dm6BWgyE46YE (disco)
['5B3gb8HbvFm02ccQVH33IO', '3owJApDCnuxVg9nw6fqlSC', '2PzF3urLEgnQs4cmokAVHX', '52G9ij1Yrh1dVmXGXqVemZ', '370aiUDCl2Dm6BWgyE46YE']


SyntaxError: invalid syntax (<ipython-input-39-f1d273a160fa>, line 1)